<a href="https://colab.research.google.com/github/RaviTejaGanti/Titanic-Survival-Prediction/blob/main/Titanic_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'titanic:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F3136%2F26502%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240221%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240221T235137Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D7eeb82e275350806ef840488ec7e532e30eb3388188aab5ea7bd31550822017c745d2d584eac87dc7aee400ba1d6bc1dd44ccbc53f6fa1d120672b51668c942220a0786de164fcf12070d864dfadef7b749c5ee3dc1e05237df0c2cdc19fface54ef815c61a97c5a572bd53a616c470aaff60eb9f73cfa51a131cdd29574e43b737a45605dcbe5e78f9cdae62a0b648b047e1ee4fc4acdcfaa56885f1cf0f57a623f24485963731669f1ef166bd88ea28564d89ab427d129e103f90993b1d92d31e846ea3d49aa99d8c2402396ecaad1c32073be1ecb13d48065087966711c3579b7f02c177e1b97d94e6b5ae83d9988c5225365b7041ee5a7c19c37f9f365f3'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns


In [ ]:
train = pd.read_csv("/kaggle/input/titanic/train.csv")
test = pd.read_csv("/kaggle/input/titanic/test.csv")

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.describe()

In [ ]:
print(train.duplicated().sum())

In [ ]:
sns.countplot(train,x='Sex',hue='Survived')

In [ ]:
sns.countplot(train,x='Pclass',hue='Survived')

In [ ]:
l = sns.FacetGrid(train,col='Survived',hue='Sex')
l.map(sns.histplot,"Age")
#sns.histplot(data = train[train['Survived']==1],x='Age')

In [ ]:
g = sns.FacetGrid(train,col='Survived')
g.map(sns.scatterplot,"Fare","Age")

In [ ]:
sns.boxplot(train,x='Survived',y='Age',hue='Sex')

In [ ]:
train.isnull().sum()

In [ ]:
train['Age'].fillna(train.Age.mean(),inplace=True)

In [ ]:
test['Age'].fillna(test.Age.mean(),inplace=True)

In [ ]:
test['Fare'].fillna(test.Fare.mean(),inplace=True)

In [ ]:
e_m = train['Embarked'].mode()[0]

train['Embarked'] = train['Embarked'].replace("", e_m)

In [ ]:
e_m_t = test['Embarked'].mode()[0]

test['Embarked'] = test['Embarked'].replace("", e_m_t)

In [ ]:
c_m = train['Cabin'].mode()[0]
train['Cabin'] = train['Cabin'].replace(np.nan, c_m)

In [ ]:
c_m_t = test['Cabin'].mode()[0]
test['Cabin'] = test['Cabin'].replace(np.nan, c_m_t)

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
l = sns.FacetGrid(train,col='Pclass',hue='Survived')
l.map(sns.countplot,"Sex")
l.add_legend()

In [ ]:
k = sns.FacetGrid(train,col='Embarked',hue='Survived')
k.map(sns.countplot,"Sex")
k.add_legend()

In [ ]:
sns.boxplot(train,x='Survived',y='Age',hue='Sex')

In [ ]:
from sklearn import preprocessing
label_encode = preprocessing.LabelEncoder()
train['Cabin'] = label_encode.fit_transform(train['Cabin'])
train['Embarked'] = label_encode.fit_transform(train['Embarked'])
train['Sex'] = label_encode.fit_transform(train['Sex'])

In [ ]:
test['Cabin'] = label_encode.fit_transform(test['Cabin'])
test['Embarked'] = label_encode.fit_transform(test['Embarked'])
test['Sex'] = label_encode.fit_transform(test['Sex'])

In [ ]:
labels = np.array(train['Survived'])
features = train.drop(['Survived','Name','Ticket'],axis=1)

In [ ]:
features_list = list(features.columns)
features = np.array(features)

In [ ]:
test = test.drop(['Name','Ticket'],axis=1)

In [ ]:
test

In [ ]:
print(test.shape)
print(features.shape)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=0)

In [ ]:
rfc.fit(features,labels)

In [ ]:
labels_pred= rfc.predict(test)

In [ ]:
output = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived':labels_pred })

In [ ]:
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")